In [225]:
import re
from nltk import Tree

def parser(expression):
    old = expression.replace(' ', '_').replace('>_(', '> (').replace(')_(', ') (').replace(')_)', ') )').replace(')_)', ') )').replace(' (', '(')
    new = ''
    flag = False
    for x in range(0, len(old) - 1):        
        if old[x] == '<':
            flag = True
        if old[x] == '>':
            flag = False
            
        if flag == True:
            if old[x] == '(':
                new += '{'
            elif old[x] == ')':
                new += '}'
            else:
                new += old[x]
        else:
            new += old[x]
    new += old[len(old)-1]
    return new

In [226]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append(pos['pos'])
    return res

In [227]:
def insert_pos_tag(exp, pos):
    count = 0
    res = ''
    for x in range(0, len(exp)):
        if exp[x] == 'S' and exp[x+1]==' ' and exp[x+2] == 'P':
            res += 'S '
            res += pos[count]
            count += 1
            x += 4
        else:
            res += exp[x]
    return res

In [228]:
def direction(exp):
    cont = False
    for x in exp:
        if x == '{':
            cont = True
        elif x == '}':
            cont = False
            continue
        if cont == True:
            continue
        if x == '/':
            return '/'
        elif x == '\\':
            return '\\'
    return False

In [229]:
def is_type_raising(tree):
    tree_string = str(tree)
    
    # check type raising
    exp = tree_string.split('_')[1]
    pattern_1 = r'(.*?)\\(.*?){(.*?)/(.*?)}'
    pattern_2 = r'(.*?)/(.*?){(.*?)\\(.*?)}'
    
    match = False
    if re.search(pattern_1, exp):
        match = True
    elif re.search(pattern_2, exp):
        match = True
        
    sub = []
    for subtree in tree:
        sub.append(subtree)
    if len(sub) == 1 and match:
        return True
    else:
        return False

In [230]:
import nltk
from nltk.sem.logic import *

read_expr = nltk.sem.Expression.fromstring

def lambda_calculus(tree):
    tree_string = str(tree)
    
    if tree_string[2] == 'L':
        pos = tree_string.split('_')[3]
        word = tree_string.split('_')[5]
        
        #                        #
        # masukin rulenya disini #
        #                        #
        if 'NN' in pos :
            return read_expr(r'(' + word + ')')
        elif 'JJ' in pos:
            return read_expr(r'(' + word + '_1)')
        elif 'VB' in pos:
            if '{S[dcl]\\NP}/{S[adj]\\NP}' in tree_string.split('_')[1]:
                return read_expr(r'\X.X')
            else:
                return read_expr(r'\X.\Y.' + word + '(X, Y)')
        else:
            return read_expr(r'\X.X')
    
    # ini cuma masukin ke array 
    chunk = []
    sub = []
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            sub.append(subtree)
            
            # chunk temp array
            subtree_str_array = str(subtree).split('_')
            if subtree_str_array[0][2] == 'L':
                if subtree_str_array[3] == 'NN':
                    chunk.append(subtree_str_array[5])
    
    
    # chunk noun phrase
    if len(chunk) == 2:
        chunk_str = '_'.join(chunk)
        return read_expr(r'(' + chunk_str + ')')
    
    # error anak 1
    if len(sub) == 1:            
        return lambda_calculus(sub[0])
                        
    # urutan operasi lambda calculusnya    
    first = sub[0]
    second = sub[1]
    
    if is_type_raising(first):
        if direction(str(first).split('_')[1]) == '/':
            x = read_expr(r'\F x.F(x, ' + str(lambda_calculus(first)) + ')')
            y = lambda_calculus(second)
            return ApplicationExpression(x, y).simplify()
    
    if is_type_raising(second):
        if direction(str(second).split('_')[1]) == '/':
            x = lambda_calculus(first)
            y = read_expr(r'\F x.F(x, ' + str(lambda_calculus(second)) + ')')
            return ApplicationExpression(x, y).simplify()
        
    length_1 = len(str(sub[0]).split('_')[1].replace('\\', '/').split('/'))
    length_2 = len(str(sub[1]).split('_')[1].replace('\\', '/').split('/'))
    if length_2 > length_1:
        first = sub[1]
        second = sub[0]
            
    
    # rekursi
    return ApplicationExpression(lambda_calculus(first), lambda_calculus(second)).simplify()

In [231]:
from_res = '(<T S[dcl] 0 2> (<T NP 0 2> (<L NP[nb]/N POS POS the NP[nb]/N>) (<L N POS POS hotel N>) ) (<T S[dcl]\\NP 0 2> (<L (S[dcl]\\NP)/NP POS POS had (S[dcl]\\NP)/NP>) (<T NP 0 2> (<L NP[nb]/N POS POS an NP[nb]/N>) (<T N 0 2> (<L N/N POS POS exceptional N/N>) (<L N POS POS service N>) ) ) ) )'
pos_tagged = insert_pos_tag(from_res, pos_tag('the hotel had a exceptional service'))

hasil = parser(pos_tagged)

from nltk import Tree
tree = Tree.fromstring(hasil)
# t.pretty_print()

In [232]:
lambda_calculus(tree)

<ApplicationExpression had(exceptional_1(service),hotel)>

In [233]:
# hasil

In [234]:
# pos_tagged

In [235]:
# print(str(tree).replace('_', ' '))

In [236]:
import requests

def glue_process(sent):
    url = "http://localhost:5000/ccgParsing"
    data = {"sent": sent}
    r = requests.post(url, data=data)

    res = r.json()
    
    from_res = res['tree']
    pos_tagged = insert_pos_tag(from_res, pos_tag(data['sent']))

    hasil = parser(pos_tagged)

    tree = Tree.fromstring(hasil)
#     print(str(tree).replace('_', ' '))
    return lambda_calculus(tree)

In [237]:
glue_process('I bought my canon g3 about a month ago and i have to say i am very satisfied')
# glue_process('the hotel had an exceptional service')

<ApplicationExpression have(\Y.say(satisfied_1(\X.X),Y),\X.X,month(\Y.bought(canon_g3,Y),\X.X))>

In [238]:
glue_process('the breakfast that the restaurant served daily was excellent')

<ApplicationExpression excellent_1(daily_1(\X Y.served(X,Y),breakfast,restaurant))>

In [239]:
glue_process('the dog bites john')

<ApplicationExpression bites(john,dog)>

In [240]:
a = read_expr(r'(restaurant)')
b = read_expr(r'((\X.served(X)))')
# ApplicationExpression(a, b).simplify()

In [241]:
x = read_expr(r'X')
restaurant = read_expr(r'restaurant')

second = read_expr(r'\X.serve(X)')
third = second(x, restaurant).simplify()
z = read_expr(r'X')

In [242]:
y = read_expr(r'\X.' + str(third))

In [224]:
# zero = read_expr(r'\F x.F(x, ' + 'res' + ')')

# one = read_expr(r'\x.\y.serve(x, y)')
# third = zero(one).simplify()
# print(zero)
# print(one)
# print(third)
# ApplicationExpression(third, read_expr(r'breakfast')).simplify()